## Digikala Customer Purchase Dashboard
### 1 - Collect orders data
### 2 - Extract order_ids
### 3 - Collect purchased products data
### 4 - Convert order date from Jalali to Gregorian 
### 5 - Clean category text
### 6 - save datas into csv

In [ ]:
# for convert Jalali date
from persiantools.jdatetime import JalaliDate

# translate data to english
from deep_translator import GoogleTranslator

# convert num unit
from unidecode import unidecode

# preprocess data
import pandas as pd

# fetch data 
import requests

## 1 - Collect orders data

In [ ]:
url = "https://api.digikala.com/v1/profile/orders/?activeTab=sent&status=sent&page=1"

purchase_history = []

payload={}


headers = {
  'authority': 'api.digikala.com',
  'accept': 'application/json, text/plain, */*',
  'accept-language': 'en-US,en;q=0.9,fa;q=0.8',
  'cookie': '',
  'origin': 'https://www.digikala.com',
  'referer': 'https://www.digikala.com/',
  'sec-ch-ua': '""',
  'sec-ch-ua-mobile': '',
  'sec-ch-ua-platform': '""',
  'sec-fetch-dest': '',
  'sec-fetch-mode': '',
  'sec-fetch-site': '',
  'user-agent': '',
  'x-web-client': '',
  'x-web-optimize-response': ''
}

response = requests.request("GET", url, headers=headers, data=payload)

data = response.json()

for result in  data['data']['orders']:
    purchase_data = {
        'id':result['id'],
        'payable_price':result['payable_price'],
        'created_at':result['created_at'],
        'total_cost':result['price_details']['total_cost'],
        'shipping_cost':result['price_details']['shipping_cost'],
        'discount':result['price_details']['discount'],   
        }
    purchase_history.append(purchase_data)   

df_orders = pd.DataFrame(purchase_history)

In [ ]:
df_orders.head()

## 2 - Extract order_ids
## 3 - Collect purchased products data¶

In [22]:
product_lists = []


# loop through order ids and fetch purchased products
for order_id in df_orders['id']:
    url = f"https://api.digikala.com/v1/order/{order_id}/?orderId={order_id}"
    payload={}
    headers = {
      'authority': 'api.digikala.com',
      'accept': 'application/json, text/plain, */*',
      'accept-language': 'en-US,en;q=0.9,fa;q=0.8',
      'cookie': '',
      'origin': 'https://www.digikala.com',
      'referer': 'https://www.digikala.com/',
      'sec-ch-ua': '""',
      'sec-ch-ua-mobile': '',
      'sec-ch-ua-platform': '""',
      'sec-fetch-dest': '',
      'sec-fetch-mode': '',
      'sec-fetch-site': '',
      'user-agent': '',
      'x-web-client': '',
      'x-web-optimize-response': ''
    }

    response_deatil = requests.request("GET", url, headers=headers, data=payload)
    data = response_deatil.json()
    for res in data['data']['order']['order_items']:
        if 'default_variant' not in res['product']:
            product_data = {
              'id':res['product']['id'],
                'order_id':order_id,
                'title_fa':res['product']['title_fa'],
                'title_en':res['product']['title_en'],
                'price':res['price']['selling_price'],
                'image_url':res['product']['images']['main']['url'][0],
                'category':res['product']['data_layer']['category']
                   }
        else :
            product_data = {
              'id':res['product']['id'],
                'order_id':order_id,
                'title_fa':res['product']['title_fa'],
                'title_en':res['product']['title_en'],
                'price':res['product']['default_variant']['price']['selling_price'],
                'image_url':res['product']['images']['main']['url'][0],
                'brand':res['product']['data_layer']['category']
                }
            
        product_lists.append(product_data)       
df_products = pd.DataFrame(product_lists)

In [6]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             8 non-null      int64 
 1   payable_price  8 non-null      int64 
 2   created_at     8 non-null      object
 3   total_cost     8 non-null      int64 
 4   shipping_cost  8 non-null      int64 
 5   discount       8 non-null      int64 
dtypes: int64(5), object(1)
memory usage: 512.0+ bytes


In [29]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         24 non-null     int64 
 1   order_id   24 non-null     int64 
 2   title_fa   24 non-null     object
 3   title_en   7 non-null      object
 4   price      24 non-null     int64 
 5   image_url  24 non-null     object
 6   brand      24 non-null     object
dtypes: int64(3), object(4)
memory usage: 1.4+ KB


In [7]:
df_orders.describe()

,id,payable_price,total_cost,shipping_cost,discount
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000e+00
mean,1.421479e+08,1.169838e+07,1.221250e+07,283625.000000,7.977500e+05
std,1.534149e+07,2.494428e+07,2.466625e+07,250035.961699,9.359173e+05
min,1.201727e+08,9.280000e+05,9.900000e+05,0.000000,0.000000e+00
25%,1.328764e+08,2.025375e+06,2.372500e+06,212250.000000,1.481250e+05
50%,1.430261e+08,2.724750e+06,3.992500e+06,235000.000000,4.710000e+05
75%,1.465450e+08,4.435000e+06,5.297500e+06,258750.000000,1.081375e+06
max,1.714989e+08,7.325400e+07,7.306500e+07,865000.000000,2.637000e+06


## 4 - Convert order date from Jalali to Gregorian

In [13]:
def convert_date(month,day,yaer):
    """
    This funciton convert persain date like "6 1401 خرداد" to gregorian "2022-5-7"
    Params:
    
    month: Jalali month name 
    day: Jalali day date
    yaer: Jalali year date like "1401"
    
    Returns:
    Converted jalali date to gregorian
    
    """
    date_dict = {1  : 'فروردین',
                 2  : 'اردیبهشت',
                 3  : 'خرداد',
                 4  : 'تیر',
                 5  : 'مرداد',
                 6  : 'شهریور',
                 7  : 'مهر',
                 8  : 'آبان',
                 9  : 'آذر',
                 10 : 'دی',
                 11 : 'بهمن',
                 12 : 'اسفند' }
    for key, value in date_dict.items():
        month = month.replace(value,str(key))
    yaer =  unidecode(yaer)
    day =  unidecode(day)
    converted_date =JalaliDate(int(yaer), int(month), int(day)).to_gregorian()
    return converted_date

In [15]:
df_orders['created_at'] = df_orders['created_at'].apply(lambda x : convert_date(x.split(' ')[1],x.split(' ')[0],x.split(' ')[2]))

## 5 - Clean category text

In [ ]:
df_products['brand'] = df_products['brand'].apply(lambda x :x.split(',')[1][:-1])

## 6 - save datas into csv

In [17]:
# save purchased products into csv file
df_orders.to_csv('orders_history.csv',index=False)

In [34]:
# save purchased products into csv file
df_products.to_csv('purchased_products.csv',index=False)